In [1]:
import pandas as pd
import numpy as np
import datetime
import numpy as np
from haversine import haversine
import json

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
excel_file=pd.ExcelFile("./BL_2020 Comp Store Closing and Opening 3.30.2020.xlsx")
excel_file.sheet_names

['Wolf Furniture',
 'Levin & Wolf 2020 Stores',
 'BBB',
 'BBB 2020 Stores',
 'BBB 2020 Planned Closings',
 "Macy's",
 "Macy's 2020 Stores",
 "Macy's 2020 Planned Closings",
 'Pier 1',
 'Pier 1 2020 Stores',
 'Pier 1 2020 Planned Closings',
 'Art Van Furniture',
 'AVF 2020 Stores',
 'AVF 2020 Planned Closings',
 'At Home',
 'AH 2020 Stores']

In [3]:
df_closed_comp_store=excel_file.parse("Wolf Furniture",dtype=str,skiprows=5)
df_closed_comp_store=df_closed_comp_store[['Company','Address','City ','State','Zip','Status']]
df_closed_comp_store=df_closed_comp_store.rename(columns={"Company":"store_name",
                                                          "Address":"address",
                                                          "City ":"city",
                                                          "State":"state",
                                                          "Zip":"zip_code",
                                                          'Status':"status"})
df_closed_comp_store['latitude']=39.3932264
df_closed_comp_store['longitude']=-77.440286
df_closed_comp_store

,store_name,address,city,state,zip_code,status,latitude,longitude
0,Wolf Furniture,5830 Ballenger Creek Pike,Frederick,MD,21703,Closed,39.393226,-77.440286


In [4]:
df_comp_stores=excel_file.parse("Levin & Wolf 2020 Stores",dtype=str)
df_comp_stores['status']='open'
df_comp_stores=df_comp_stores[df_closed_comp_store.columns.tolist()]

In [5]:
df_all_wolf_and_levin_stores=df_closed_comp_store.append(df_comp_stores)
print(df_all_wolf_and_levin_stores.shape,df_all_wolf_and_levin_stores['store_name'].nunique())
print("df_all_wolf_and_levin_stores['zip_code'].apply(len).unique()",df_all_wolf_and_levin_stores['zip_code'].apply(len).unique())
df_all_wolf_and_levin_stores['latitude']=df_all_wolf_and_levin_stores['latitude'].astype(float)
df_all_wolf_and_levin_stores['longitude']=df_all_wolf_and_levin_stores['longitude'].astype(float)
df_all_wolf_and_levin_stores=df_all_wolf_and_levin_stores.reset_index()
del df_all_wolf_and_levin_stores['index']

df_all_wolf_and_levin_stores.head(2)

(57, 8) 57
df_all_wolf_and_levin_stores['zip_code'].apply(len).unique() [5]


,store_name,address,city,state,zip_code,status,latitude,longitude
0,Wolf Furniture,5830 Ballenger Creek Pike,Frederick,MD,21703,Closed,39.393226,-77.440286
1,Levin - Montrose,3742 Brookwall Drive,Akron,OH,44333,open,41.131590,-81.644194


In [6]:
df_competitor_zips_20=pd.DataFrame()
for ind,row in df_all_wolf_and_levin_stores.iterrows():
    store_center=(row['latitude'],row['longitude'])
    store_name=row['store_name']
    store_zip_list_20=[]
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_center,v,unit="mi")
        if dist<=20:
            store_zip_list_20.append(zip_cd)
    df=pd.DataFrame({"store_name":[store_name]*len(store_zip_list_20),"zip_cd":store_zip_list_20})
    df_competitor_zips_20=df_competitor_zips_20.append(df)
df_competitor_zips_20.shape

(5353, 2)

In [7]:
df_competitor_zips_20.head(2)

,store_name,zip_cd
0,Wolf Furniture,20129
1,Wolf Furniture,21720


In [8]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200401-135137-445.txt",
                            dtype=str,sep="|")
df_store_list=df_store_list[['location_id','latitude_meas','longitude_meas']]
df_store_list=df_store_list[~df_store_list['location_id'].isin(['145','6990'])]
df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)
'''
for col in df_store_list.columns.tolist():
    df_store_list=df_store_list.rename(columns={col:"BL_"+col})
'''
df_store_list.head(3)

,location_id,latitude_meas,longitude_meas
0,3,33.421570,-82.018921
1,30,41.363132,-83.650909
2,43,41.612553,-83.623582


In [9]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
#
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)
    
#   
df_output=pd.merge(df_store_list,df_zip_by_store,on="location_id",how="left")
df_output_2=df_output[pd.isnull(df_output['zip_cd'])]
df_output_1=df_output[pd.notnull(df_output['zip_cd'])]
print(df_output_2.shape)

del df_output_2['zip_cd']
del df_output_2['zip_type']
#

df_10_zips_all_new_stores=pd.DataFrame()
for ind, row in df_output_2.iterrows():
    location_id=row['location_id']
    store_location=(row['latitude_meas'],row['longitude_meas'])
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_location,v,unit="mi")
        if dist<=10:
            # print(location_id,zip_cd)
            df=pd.DataFrame({"location_id":[location_id],"zip_cd":[zip_cd],"dist":[dist]})
            df_10_zips_all_new_stores=df_10_zips_all_new_stores.append(df)
df_10_zips_all_new_stores['zip_type']="zip_10"
print(df_10_zips_all_new_stores['location_id'].nunique())
#
df_new_store_zips=df_10_zips_all_new_stores[['location_id','zip_cd','zip_type']]
df_output_2=pd.merge(df_output_2,df_new_store_zips,how="left")
df_output=df_output_1.append(df_output_2)

(34, 5)
34


In [10]:
df_output['zip_type']=df_output['zip_type'].replace("zip_10","P")
df_output_P=df_output[df_output['zip_type']=="P"]
df_output_P=df_output_P[['location_id','zip_cd','zip_type']].drop_duplicates()

In [11]:
df_all_wolf_and_levin_stores_paired_P=pd.merge(df_all_wolf_and_levin_stores,df_output_P,left_on="zip_code",right_on="zip_cd",how="left")
df_all_wolf_and_levin_stores_paired_P=df_all_wolf_and_levin_stores_paired_P[df_all_wolf_and_levin_stores_paired_P['zip_type']=="P"]
del df_all_wolf_and_levin_stores_paired_P['zip_cd']
df_all_wolf_and_levin_stores_paired_P=df_all_wolf_and_levin_stores_paired_P.rename(columns={"zip_type":"paired_condition"})
print(df_all_wolf_and_levin_stores_paired_P.shape,df_all_wolf_and_levin_stores_paired_P['store_name'].nunique())
df_all_wolf_and_levin_stores_paired_P.head(2)

(51, 10) 46


,store_name,address,city,state,zip_code,status,latitude,longitude,location_id,paired_condition
0,Wolf Furniture,5830 Ballenger Creek Pike,Frederick,MD,21703,Closed,39.393226,-77.440286,5395,P
3,Levin - Boardman - Now Open!,300 Boardman Poland Road,Boardman,OH,44512,open,41.025035,-80.651381,1466,P


In [12]:
df_paired_20=pd.DataFrame()
for ind,row in df_all_wolf_and_levin_stores.iterrows():
    store_center=(row['latitude'],row['longitude'])
    store_name=row['store_name']
    BL_store_list_20=[]
    for ind_BL,row_BL in df_store_list.iterrows():
        BL_store_location=row_BL['location_id']
        BL_store_center=(row_BL['latitude_meas'],row_BL['longitude_meas'])
        dist=haversine(store_center,BL_store_center,unit="mi")
        if dist<=20:
            BL_store_list_20.append(BL_store_location)
    df=pd.DataFrame({"store_name":[store_name]*len(BL_store_list_20),"location_id":BL_store_list_20})
    df_paired_20=df_paired_20.append(df)
print(df_paired_20.shape,df_paired_20['store_name'].nunique())

(357, 2) 57


In [13]:
df_all_wolf_and_levin_stores_paired_20=pd.merge(df_all_wolf_and_levin_stores,df_paired_20,on="store_name",how="left")
df_all_wolf_and_levin_stores_paired_20['paired_condition']="dist_20"
df_all_wolf_and_levin_stores_paired=df_all_wolf_and_levin_stores_paired_P.append(df_all_wolf_and_levin_stores_paired_20)

In [14]:
df_store_list=df_store_list.rename(columns={"latitude_meas":"BL_latitude_meas","longitude_meas":"BL_longitude_meas"})
df_all_wolf_and_levin_stores_paired=pd.merge(df_all_wolf_and_levin_stores_paired,df_store_list,on="location_id",how="left")

In [15]:
df_all_wolf_and_levin_stores_paired['store_dist']=df_all_wolf_and_levin_stores_paired.apply(lambda x: haversine([x['latitude'],x['longitude']],[x['BL_latitude_meas'],x['BL_longitude_meas']],unit='mi'),axis=1)
df_all_wolf_and_levin_stores_paired.shape

(408, 13)

In [16]:
df_competitor_zips_20=df_competitor_zips_20.rename(columns={"zip_cd":"comp_target_zip"})

df_output_detail=pd.merge(df_all_wolf_and_levin_stores_paired,df_competitor_zips_20,on="store_name",how="left")

df_output_detail.shape

(43730, 14)

In [17]:
df_output_detail=df_output_detail.reset_index()
del df_output_detail['index']
df_output_detail['dist_comp_zip_to_BL_store']=df_output_detail.apply(lambda x: haversine((x['BL_latitude_meas'],x['BL_longitude_meas']),zip_centers[x['comp_target_zip']],unit="mi"),axis=1)
print(df_output_detail.shape)
df_output_detail.head(2)

(43730, 15)


,store_name,address,city,state,zip_code,status,latitude,longitude,location_id,paired_condition,BL_latitude_meas,BL_longitude_meas,store_dist,comp_target_zip,dist_comp_zip_to_BL_store
0,Wolf Furniture,5830 Ballenger Creek Pike,Frederick,MD,21703,Closed,39.393226,-77.440286,5395,P,39.417166,-77.456782,1.873905,20129,19.648582
1,Wolf Furniture,5830 Ballenger Creek Pike,Frederick,MD,21703,Closed,39.393226,-77.440286,5395,P,39.417166,-77.456782,1.873905,21720,17.086080


In [18]:
df_output=df_output_detail[df_output_detail['dist_comp_zip_to_BL_store']<=30]
print(df_output.shape)
df_output.head(2)

(41734, 15)


,store_name,address,city,state,zip_code,status,latitude,longitude,location_id,paired_condition,BL_latitude_meas,BL_longitude_meas,store_dist,comp_target_zip,dist_comp_zip_to_BL_store
0,Wolf Furniture,5830 Ballenger Creek Pike,Frederick,MD,21703,Closed,39.393226,-77.440286,5395,P,39.417166,-77.456782,1.873905,20129,19.648582
1,Wolf Furniture,5830 Ballenger Creek Pike,Frederick,MD,21703,Closed,39.393226,-77.440286,5395,P,39.417166,-77.456782,1.873905,21720,17.086080


In [19]:
df_zips_with_stores=df_output[['store_name','location_id','comp_target_zip']].drop_duplicates()
df_unique_zips=df_output[['comp_target_zip']].drop_duplicates()

In [26]:
df_SOTF_list=pd.ExcelFile("./Store List Report 03.06.20 409pm.xlsx")
df_SOTF_list=df_SOTF_list.parse("Store List",dtype=str)
df_SOTF_list=df_SOTF_list[['Store','SOTF']]
df_SOTF_list=df_SOTF_list[pd.notnull(df_SOTF_list['Store'])]
df_SOTF_list=df_SOTF_list.rename(columns={"Store":"location_id"})

In [29]:
df_BL_store_overlappped=df_output[['location_id']].drop_duplicates()
df_BL_store_overlappped=pd.merge(df_BL_store_overlappped,df_SOTF_list,on="location_id",how="left")
df_BL_store_overlappped['SOTF']=df_BL_store_overlappped['SOTF'].replace({"No":"Legacy","Yes":"SOTF","Yes 2020":"SOTF 2020"})
df_BL_store_overlappped.shape

(87, 2)

In [32]:
df_BL_store_summary=df_BL_store_overlappped.groupby("SOTF")['location_id'].count().to_frame().reset_index()
df_BL_store_summary

,SOTF,location_id
0,Legacy,22
1,SOTF,45
2,SOTF 2020,20


In [33]:
writer=pd.ExcelWriter("./BL_zips_for_Wolf_and_Levin_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zips.to_excel(writer,"unique_zips",index=False)

df_BL_store_summary.to_excel(writer,"BL_store_summary",index=False)
df_BL_store_overlappped.to_excel(writer,"BL_store_list",index=False)

df_zips_with_stores.to_excel(writer,"zips_by_store",index=False)
df_output.to_excel(writer,"detail",index=False)
writer.save()

In [25]:
df_unique_zips.shape

(1579, 1)

In [26]:
df_output['comp_target_zip'].nunique()

1579

In [28]:
import os
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2020_Q1/Competitor_conquest'